# Hyperbolic tangent activation and dense network


### *Import some libraries*

In [1]:
import numpy as np
from math import sin
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### *Define a model*

In this case we are using a bipolar function that is continuous, monotone and bounded, which is to say it is a universal approximator.

In [23]:
model = Sequential([
    Dense(50, input_dim=1, activation='tanh', use_bias=True),
    Dense(50, activation='tanh', use_bias=True),
    Dense(50, activation='tanh', use_bias=True),
    Dense(1, activation='tanh', use_bias=True)
])
model.compile(optimizer='adam', loss='mse')
model.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 50)                100       
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 51        
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________


### *Fit model with a generator*

In [24]:
def gen_batch(batch_size):
    X = []
    Y = []
    for i in range(10):
        x = np.random.uniform(-np.pi/2, np.pi/2)
        y = sin(x)
        X.append(x)
        Y.append(y)
    return np.array([X, Y])

def gen(batch_size):
    while 1:
        yield gen_batch(batch_size)
        
model.fit_generator(generator=gen(20), steps_per_epoch=10000, epochs=10, validation_data=gen(5), validation_steps=10)

Epoch 1/10
10000/10000 [==============================] - 14s - loss: 8.0505e-04 - val_loss: 2.1342e-04
Epoch 2/10
10000/10000 [==============================] - 14s - loss: 1.8147e-04 - val_loss: 1.5821e-04
Epoch 3/10
10000/10000 [==============================] - 14s - loss: 1.4495e-04 - val_loss: 7.8119e-05
Epoch 4/10
10000/10000 [==============================] - 14s - loss: 7.0941e-05 - val_loss: 1.8047e-05
Epoch 5/10
10000/10000 [==============================] - 14s - loss: 3.4249e-05 - val_loss: 1.8572e-05
Epoch 6/10
10000/10000 [==============================] - 14s - loss: 2.4468e-05 - val_loss: 1.0710e-05
Epoch 7/10
10000/10000 [==============================] - 14s - loss: 2.2358e-05 - val_loss: 3.6598e-06
Epoch 8/10
10000/10000 [==============================] - 14s - loss: 1.9545e-05 - val_loss: 7.2778e-06
Epoch 9/10
10000/10000 [==============================] - 14s - loss: 1.3780e-05 - val_loss: 1.8593e-05
Epoch 10/10
10000/10000 [==============================] - 13s -

### *Validate against original function*

In [25]:
x = [float(x-15)/10 for x in range(30)]
y = model.predict(np.array(x))
z = [sin(i) for i in x]

q = zip(x,[i[0] for i in y.tolist()],z)
for p in q:
    if p[2] != 0:
        print(p, abs(p[1] - p[2]) / abs(p[2]))

((-1.5, -0.9940923452377319, -0.9974949866040544), 0.0034111864340358395)
((-1.4, -0.984131932258606, -0.9854497299884601), 0.0013372551534106343)
((-1.3, -0.9617215991020203, -0.963558185417193), 0.0019060460934982413)
((-1.2, -0.9290411472320557, -0.9320390859672263), 0.0032165375683354575)
((-1.1, -0.8879474401473999, -0.8912073600614354), 0.0036578691560747337)
((-1.0, -0.8381936550140381, -0.8414709848078965), 0.003894762687042164)
((-0.9, -0.7796767950057983, -0.7833269096274834), 0.004659759005880331)
((-0.8, -0.7130698561668396, -0.7173560908995228), 0.00597504473309553)
((-0.7, -0.6395702958106995, -0.644217687237691), 0.007214007809873821)
((-0.6, -0.5603370666503906, -0.5646424733950354), 0.007625013964602334)
((-0.5, -0.4759567975997925, -0.479425538604203), 0.007235202810658354)
((-0.4, -0.3864668011665344, -0.3894183423086505), 0.0075793582927245005)
((-0.3, -0.29267024993896484, -0.29552020666133955), 0.009643864135628118)
((-0.2, -0.19765667617321014, -0.198669330795061

The results aren't that bad.  However, in general, the greater the slope, the worse the performance. The degree depends on the function being approximated and the activation function.  For example, sin^2 is better approximated by a sigmoid than tanh, which works since sigmoid is a universal approximator and is strictly positive.

Since we are interested in areas near singularities for inverse kinematics, the problem with accuracy at higher gradients is a problem.  One solution is to use more neurons, but the number of neurons grows quickly with the size of the work area and the gradient of the manifold we are approximating.  Another option is to use a significantly different type of activation function having a geometric aspect, such as radial basis functions. 

# Radial Basis Functions

### *Import some libraries*

In [6]:
import numpy as np
from keras import backend as K
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomUniform, Initializer, Orthogonal, Constant

### *Define the custom layer* 
(Idea taken from https://github.com/PetraVidnerova/rbf_keras)


In [14]:
class RBFLayer(Layer):
   
    def __init__(self, output_dim, initializer=RandomUniform(0.0, 1.0), betas=1.0, **kwargs):
        self.output_dim = output_dim
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.centers = self.add_weight(name        = 'centers',
                                       shape       = (self.output_dim, input_shape[1]),
                                       initializer = RandomUniform(0.0, 1.0),
                                       trainable   = True)
        self.betas = self.add_weight(name        = 'betas',
                                     shape       = (self.output_dim),
                                     initializer = Constant(value=1.0),
                                     trainable   = True)
        super(RBFLayer, self).build(input_shape)  

    def call(self, x):
        C = K.expand_dims(self.centers)
        H = (C-x.T).T
        return K.exp(-self.betas * K.sum(H**2, axis=1))
   
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### *Define an initializer*

In [8]:
class InitCentersRandom(Initializer):
  
    def __init__(self, X):
        self.X = X 

    def __call__(self, shape, dtype=None):
        print(shape)
        print(self.X.shape)
        assert shape[1] == self.X.shape[1]
        idx = np.random.randint(self.X.shape[0], size=shape[0])
        return self.X[idx,:]

### *Define basis and model*

In [15]:
X = np.array(zip([float(x-5)/10 for x in range(10)],[float(x-5)/10 for x in range(10)]))
  
model = Sequential([
    RBFLayer(10, initializer=InitCentersRandom(X), betas=1.0, input_shape=(1,)),
    Dense(1)
])

model.fit_generator(generator=gen(20), steps_per_epoch=10000, epochs=10, validation_data=gen(5), validation_steps=10)

TypeError: 'int' object is not iterable

### *Validate against original function*

In [ ]:
x = [float(x-15)/10 for x in range(30)]
y = model.predict(np.array(x))
z = [sin(i) for i in x]

q = zip(x,[i[0] for i in y.tolist()],z)
for p in q:
    if p[2] != 0:
        print(p, abs(p[1] - p[2]) / abs(p[2]))

# Another Candidate Way of Dealing With RBF

In [19]:
# https://stackoverflow.com/questions/40832066/keras-tensorflow-valueerror-shape-12-must-have-rank-1
# https://github.com/fchollet/keras/issues/7362

l2_norm = lambda a, b: K.sqrt(((a - b) ** 2).sum())

def rbf(x, gamma=1.0):
    return K.exp(-1 * gamma * l2_norm(x[0, :], x[1, :]) ** 2)
    
X = np.array(zip([float(x-5)/10 for x in range(10)],[float(x-5)/10 for x in range(10)]))
  
model = Sequential([
    Dense(10, input_dim=1, activation='tanh', use_bias=True),
    Dense(50, activation=rbf, use_bias=True),
    Dense(1, activation='tanh')
])

model.fit_generator(generator=gen(20), steps_per_epoch=10000, epochs=10, validation_data=gen(5), validation_steps=10)

AttributeError: 'Tensor' object has no attribute 'sum'